In [1]:
#connect to google drive
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [28]:
#load correlation matrix
import pickle
import numpy as np
all_corr = pickle.load(open('/content/drive/My Drive/correlation_matrix/correlations_filecc200sa.pkl', 'rb'))

In [29]:
print((all_corr['Caltech_0051456'][0]))
print((all_corr['Caltech_0051456'][1]))
print((all_corr['Caltech_0051456'][2]))
print((all_corr['Caltech_0051456'][3]))

[-0.18475538  0.0419445  -0.12917446 ...  0.20151634  0.20050212
  0.00984184]
1
1
55.4


In [30]:
#prepair dataset as input for NN model
import pandas as pd
from sklearn.model_selection import train_test_split

import tensorflow as tf
from keras import Sequential
from keras.layers import Dense
from sklearn.metrics import confusion_matrix
from sklearn.preprocessing import StandardScaler

allcorr = all_corr

all_corr = dict(list(allcorr.items())[len(allcorr)//10:])
rest = dict(list(allcorr.items())[:len(allcorr)//10])

X = []
for ff in all_corr:
  X.append(list(all_corr[ff][0]))
X = pd.DataFrame(X)
sc = StandardScaler()
X = sc.fit_transform(X)
Y = []
for jj in all_corr:
  Y.append(all_corr[jj][1])
Y = np.asarray(Y)

print(np.shape(X))
print(np.shape(Y))

(796, 19900)
(796,)


In [31]:
S = []
for ll in rest:
  S.append(rest[ll][2])
S = np.asarray(S)

A = []
for nn in rest:
  A.append(rest[nn][3])
A = np.asarray(A)

print(np.shape(S))
print(np.shape(A))

(88,)
(88,)


In [32]:
#make a NN model
import keras
classifier = Sequential()
classifier.add(Dense(1990, activation='relu', kernel_initializer='random_normal', input_dim=19900))
# classifier.add(Dense(13, activation='relu', kernel_initializer='random_normal'))
# classifier.add(Dense(2, activation='relu', kernel_initializer='random_normal'))
classifier.add(Dense(1, activation='sigmoid', kernel_initializer='random_normal'))
loss_fn = tf.keras.losses.MeanSquaredError(reduction='sum_over_batch_size')

classifier.compile(optimizer ='adam',loss=loss_fn, metrics =['accuracy'])

accuracy = []
F1 = []
x_train = X
y_train = Y
# x_train, x_test, y_train, y_test = train_test_split(X, Y, test_size=0.10)

In [33]:
#fit model
classifier.fit(x_train,y_train, batch_size=40, epochs= 100)

Epoch 1/100
20/20 [==============================] - 0s 13ms/step - loss: 0.4481 - accuracy: 0.5440
Epoch 2/100
20/20 [==============================] - 0s 12ms/step - loss: 0.3461 - accuracy: 0.6533
Epoch 3/100
20/20 [==============================] - 0s 11ms/step - loss: 0.3304 - accuracy: 0.6683
Epoch 4/100
20/20 [==============================] - 0s 11ms/step - loss: 0.3332 - accuracy: 0.6658
Epoch 5/100
20/20 [==============================] - 0s 11ms/step - loss: 0.3317 - accuracy: 0.6671
Epoch 6/100
20/20 [==============================] - 0s 11ms/step - loss: 0.3178 - accuracy: 0.6809
Epoch 7/100
20/20 [==============================] - 0s 11ms/step - loss: 0.3076 - accuracy: 0.6922
Epoch 8/100
20/20 [==============================] - 0s 11ms/step - loss: 0.2998 - accuracy: 0.6997
Epoch 9/100
20/20 [==============================] - 0s 11ms/step - loss: 0.2988 - accuracy: 0.6997
Epoch 10/100
20/20 [==============================] - 0s 11ms/step - loss: 0.3019 - accuracy: 0.6972

In [34]:
Men = {}
Women = {}
for item in rest:
  if(rest[item][2] == 1):
    Men[item] = rest[item]
  else:
    Women[item] = rest[item]

In [46]:
len(Women)
# Men

12

In [47]:
Xmen = []
for ff in Men:
  Xmen.append(list(Men[ff][0]))
Xmen = pd.DataFrame(Xmen)
sc = StandardScaler()
Xmen = sc.fit_transform(Xmen)
Ymen = []
for jj in Men:
  Ymen.append(Men[jj][1])
Ymen = np.asarray(Ymen)

In [48]:
np.shape(Ymen)

(76,)

In [49]:
y_pred=classifier.predict(Xmen)

y_pred = np.where(y_pred > 0.5, 1, 0)

cm = confusion_matrix(Ymen, y_pred)
accuracy.append((cm[0][0] + cm[1][1])/sum(sum(cm)))
precision = cm[0][0]/(cm[0][0] + cm[1][1])
recall =  cm[0][0]/(cm[0][0] + cm[1][0])
F1.append(2 * precision * recall / (precision + recall))

print('accuracy =', np.mean(accuracy))
print('F1 =', np.mean(F1))

accuracy = 0.7105263157894737
F1 = 0.577319587628866


In [ ]:
#calculate accuracy & f1 in test dataset
y_pred=classifier.predict(x_test)

y_pred = np.where(y_pred > 0.5, 1, 0)

cm = confusion_matrix(y_test, y_pred)
accuracy.append((cm[0][0] + cm[1][1])/sum(sum(cm)))
precision = cm[0][0]/(cm[0][0] + cm[1][1])
recall =  cm[0][0]/(cm[0][0] + cm[1][0])
F1.append(2 * precision * recall / (precision + recall))

print('accuracy =', np.mean(accuracy))
print('F1 =', np.mean(F1))

accuracy = 0.5955056179775281
F1 = 0.5203934642136888


In [ ]:
#get weights from model
for layer in classifier.layers:
    weights = layer.get_weights() # list of numpy arrays

In [ ]:
# np.shape(classifier.layers[0].get_weights()[1])